In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# Transformasi Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
# Load CIFAR-10 Dataset
batch_size = 32
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Split train dataset menjadi train dan validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# CNN Arsitektur
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling_type='max'):
        super(CNN, self).__init__()
        padding = kernel_size // 2
        self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=padding)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=padding)

        if pooling_type == 'max':
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pooling_type == 'avg':
            self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
# Training dan Evaluasi
def train_and_evaluate(kernel_size, pooling_type, optimizer_type, epochs, early_stopping_patience=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNN(kernel_size=kernel_size, pooling_type=pooling_type).to(device)
    criterion = nn.CrossEntropyLoss()

    # Optimizer
    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif optimizer_type == 'RMSProp':
        optimizer = optim.RMSprop(model.parameters(), lr=0.01)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Learning Rate Scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)

    best_loss = float('inf')
    patience_counter = 0

    print(f"Evaluating with params: {{'epochs': {epochs}, 'kernel_size': {kernel_size}, 'optimizer_type': '{optimizer_type}', 'pooling_type': '{pooling_type}'}}")

    for epoch in range(epochs):
        # Training Step
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)

        # Validation Step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        scheduler.step(val_loss)

        # Early Stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print("Early stopping triggered.")
                break

    # Evaluasi Akurasi pada Test Dataset
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# Eksperimen dengan Semua Kombinasi Parameter
kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
optimizers = ['SGD', 'RMSProp', 'Adam']
epoch_list = [5, 50, 100, 250, 350]

results = []

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer in optimizers:
            for epochs in epoch_list:
                print(f"Running experiment with kernel_size={kernel_size}, pooling_type={pooling_type}, optimizer={optimizer}, epochs={epochs}")
                accuracy = train_and_evaluate(kernel_size, pooling_type, optimizer, epochs, early_stopping_patience=10)
                results.append((kernel_size, pooling_type, optimizer, epochs, accuracy))

# Menampilkan Hasil Akhir
print("\nHasil Akhir Eksperimen:")
for result in results:
    print(f"Kernel Size: {result[0]}, Pooling: {result[1]}, Optimizer: {result[2]}, Epochs: {result[3]}, Accuracy: {result[4]:.2f}%")

Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=5
Evaluating with params: {'epochs': 5, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}
Epoch 1/5, Loss: 1.6535, Val Loss: 1.2997
Epoch 2/5, Loss: 1.2509, Val Loss: 1.0239
Epoch 3/5, Loss: 1.0786, Val Loss: 0.9654
Epoch 4/5, Loss: 0.9817, Val Loss: 0.9008
Epoch 5/5, Loss: 0.8940, Val Loss: 0.9055
Accuracy: 69.18%
Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=50
Evaluating with params: {'epochs': 50, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}
Epoch 1/50, Loss: 1.6518, Val Loss: 1.2830
Epoch 2/50, Loss: 1.2485, Val Loss: 1.1055
Epoch 3/50, Loss: 1.0867, Val Loss: 0.9638
Epoch 4/50, Loss: 0.9748, Val Loss: 0.9195
Epoch 5/50, Loss: 0.8919, Val Loss: 0.8581
Epoch 6/50, Loss: 0.8157, Val Loss: 0.8519
Epoch 7/50, Loss: 0.7554, Val Loss: 0.8516
Epoch 8/50, Loss: 0.7017, Val Loss: 0.8948
Epoch 9/50, Loss: 0.6514, Val Loss: 0.8833
Epoch 10/50, L